In [1]:
from google.colab import drive,userdata

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install streamlit -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 507.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.9 MB/s eta 0:00:00


In [4]:
!pip install PyPDF2 -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [5]:
!pip install langchain_community -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00


In [6]:
!pip install langchain_google_genai -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00


In [7]:
!pip install python-docx -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.5 MB/s eta 0:00:00


In [8]:
!pip install docx2txt -qq

  Preparing metadata (setup.py) ... done


In [9]:
!pip install faiss-gpu -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00


In [10]:
!pip install google-generativeai -qq


In [11]:
!pip install --upgrade google-generativeai -qq


In [12]:
import docx2txt
def extract_text(file_path):
    if file_path.endswith(".docx"):
        # Extract text from DOCX file
        return docx2txt.process(file_path)

    elif file_path.endswith(".pdf"):
        # Extract text from PDF file
        text = ""
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
        return text

    else:
        raise ValueError("Unsupported file type")

In [13]:
# from google.colab import auth
# auth.authenticate_user()


In [14]:

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Resume/firm-capsule-436804-b5-5f553d9f1043.json"


In [15]:
# !pip install python-docx

In [16]:
import os
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from google.colab import drive
from docx import Document
import google.generativeai as genai
from datetime import datetime

api_key_google = userdata.get('google_cloud')
genai.configure(api_key=api_key_google)

# Mount Google Drive
drive.mount('/content/drive')

model = genai.GenerativeModel('gemini-pro')





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
def save_resume_to_docx(tailored_resume, file_path):
    doc = Document()
    doc.add_heading('Tailored Resume', level=1)
    doc.add_paragraph(tailored_resume)
    doc.save(file_path)


In [18]:
# Function to read text from a .docx file
def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

In [19]:
def generate_resume_text(resume_text):
    prompt = f"""
Given the following resume content:

[Resume Start]
{resume_text}
[Resume End]

Format this resume content with appropriate section titles. Only use the information provided and avoid placeholders like "[Your Name]". Ensure it retains the structure and details exactly as shown.
"""
    try:
        response = model.generate_content(prompt)
        print(response)
        # Accessing the generated text content
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        print("Error in generating resume text:", e)
        return None

In [20]:
def tailor_resume(resume_text, job_description):
    # Use the generate_resume_text function to get the formatted resume content
    formatted_resume = generate_resume_text(resume_text)
    print("formatted resume:",formatted_resume)
    if formatted_resume:
        prompt = f"""
Below is the candidate's original formatted resume content:

[Resume Start]
{formatted_resume}
[Resume End]

Using the candidate's resume above and the job description below, create a tailored resume.

[Job Description Start]
{job_description}
[Job Description End]

Please generate a resume that:
1. Uses real data from the candidate's resume, including name, education, and specific skills.
2. Avoids placeholders like "[Your Name]" and includes actual details.
3. Emphasizes experiences that are directly relevant to the job description.
"""
        try:
            response = model.generate_content(prompt)
            return response.candidates[0].content.parts[0].text
        except Exception as e:
            print("Error in tailoring resume:", e)
            return None
    else:
        return "Failed to generate resume text."

In [21]:
# Main function for Streamlit app
def main():
    st.header('Resume Tailoring')

    # Load the resume and job description from Google Drive
    resume_text = read_docx("/content/drive/MyDrive/Resume/Anushka_Bhat_latest.docx")
    job_description = read_docx("/content/drive/MyDrive/Resume/Job_description.docx")

    # Tailor resume based on job description
    tailored_resume = tailor_resume(resume_text, job_description)
    st.write("**Tailored Resume:**")
    st.write(tailored_resume)
    print(tailored_resume)

        # Save the tailored resume to a .docx file
    if tailored_resume:
        file_path = f"Tailored_Resume_{datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
        save_resume_to_docx(tailored_resume, file_path)
        st.success(f"Tailored resume saved to {file_path}")

In [22]:
if __name__ == '__main__':
    main()

2024-11-05 01:56:12.910 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:13.570 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-05 01:56:13.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:22.835 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 6303.59ms


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "**Anushka Bhat**\n\n**Contact Information**\n* Email: anushkabhat9@gmail.com\n* LinkedIn: anushkabhat07\n* Phone: 1-260-515-4301\n\n**Summary**\nOver 7.5 years of experience in Database and Business Intelligence applications development. Currently pursuing a Master's in Computer Science at Purdue University Fort Wayne, specializing in Machine Learning and Deep Learning.\n\n**Education**\n**Master of Science in Computer Science**\nPurdue University Fort Wayne, Fort Wayne, IN\n* Machine Learning, Natural Language Processing, Software Engineering, Web Development, Database Management Systems, Corporate Partners\n* GPA: 4.0\n* 2023-2025\n\n**Bachelor of Engineering in Computer Science**\nFirst Class\n2011-2015\n\n**Languages and Technologies**\n* Azure SQL, T-SQ

2024-11-05 01:56:29.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 01:56:29.865 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.

**Anushka Bhat**

**Contact Information**
* Email: anushkabhat9@gmail.com
* LinkedIn: anushkabhat07
* Phone: 1-260-515-4301

**Summary**
Engineering student with 7.5+ years of experience in manufacturing process improvement, problem-solving, and data analysis. Proven ability to identify and implement cost-saving initiatives.

**Education**
* **Master of Science in Computer Science**
Purdue University Fort Wayne, Fort Wayne, IN
* Specialization in Machine Learning and Deep Learning
* GPA: 4.0
* 2023-2025

* **Bachelor of Engineering in Computer Science**
First Class
2011-2015

**Skills**
* Process Improvement (Lean Manufacturing, Cycle Time Reduction)
* Problem-Solving and Troubleshooting
* Statistical Analysis and Data Visualization
* Manufacturing Equipment Installation and Upgrades
* MS Office Suite
* Python, HTML, CSS, JavaScript

**Experience**
* **Tax Senior**
Ernst & Young GDS, India
* Sept 2019 - Aug 2023
* Implemented cost-saving measures by automating data extraction and analy